In [15]:
using Pkg
using Distributions

using Optim
using PyPlot
using LinearAlgebra

In [23]:
function market_shares(p,alpha)
    return exp.(-alpha.*p) / (1 + sum( exp.(-alpha.*p) ) )
end


function mco_profits(p,phi,mu,alpha)
    return market_shares(p,alpha) .* ( p - mu .* phi )
end


function price_profits(phi,mu,alpha)
    diff =  5.
    p = [1.]
    profits = [0.]
    
    #set up objective
    obj1(p0) = -1*mco_profits( [p0[1] ], phi,mu,alpha)[1]
    res1 = optimize(obj1, [p[1]] , BFGS())

    #update result
    p = res1.minimizer[1]
    profits =  [-1*res1.minimum]        
    return p, profits
end


#arbitrary outside option... passive beliefs
function nash_in_nash(phi, mu, alpha; mc=0., pi0=0.)
    p, profits = price_profits(phi,mu,alpha)
    s = market_shares(p,alpha)
    return  profits .* (sum(mu .* s .* (phi .- mc) ) - pi0)
end


function simult_bargain(mu, alpha; mc=0.)
    """solve each firm 1s optimization holding theta 2 fixed"""
    phi = alpha
    obj1(phi0) = -1*nash_in_nash( [phi0[1] ] , mu, alpha ; mc=mc)[1]
    res1 =  optimize(obj1, [phi[1]], BFGS())

    return [ res1.minimizer[1] ]
end

println( simult_bargain([2.],[.5]) )

print( mco_profits([3],[3],[.5],.1) )
print( nash_in_nash( [3], [.5] , .1 ) )

[0.638336][0.73532]

In [30]:
function make_table(steps, mu)
    """the argument is 
    steps = min_arg, max_arg, no_step """

    min_arg, max_arg, no_step = steps    
    step_size = (max_arg - min_arg)/no_step
    indep_var = min_arg:step_size:max_arg
    result1 = []

    for alpha=indep_var

        phi = [0.]
        phi = simult_bargain(mu, alpha)
        price, profits = price_profits(phi,mu,alpha)
        share = market_shares(price, alpha)
        hosp = sum(mu .* share .* phi)

        append!(result1, [[alpha, phi[1], share[1], price[1], profits[1], hosp ]] ) 
    end
    
    println("\n")
    for line =result1
        print( round(line[1],sigdigits=3) )
        for i=2:6
            print( " & ")
            print(round(line[i],sigdigits=3) )
        end
        print(" \\\\\\\\ \n")
    end   
    print('\n')
        
   
    

end

    
make_table( (.5,1,5), [.5] )



0.5 & 2.51 & 0.143 & 3.59 & 0.333 & 0.179 \\\\ 
0.6 & 2.09 & 0.143 & 2.99 & 0.277 & 0.149 \\\\ 
0.7 & 1.79 & 0.143 & 2.56 & 0.238 & 0.128 \\\\ 
0.8 & 1.57 & 0.143 & 2.24 & 0.208 & 0.112 \\\\ 
0.9 & 1.4 & 0.143 & 1.99 & 0.185 & 0.0995 \\\\ 
1.0 & 1.26 & 0.143 & 1.79 & 0.166 & 0.0895 \\\\ 

